In [ ]:
# HDF5 Dataset Overview
'''
This notebook provides an interface to the HDF5 file containing numor data extracted from GRASP. 
Each 'Numor' group in the HDF5 file consists of datasets including `q_x`, `q_y`, `intensity`, and `intensity_err`, 
along with metadata such as `count_time`, magnetic field, heating power, temperature sensors readings, etc. 
''' 

## Function Overview

'''
### plot_numor_data
**Purpose:** Visualize scattering data for individual numors.  
**Usage:** Ideal for initial data exploration and quality checks on data collection.  
**Example:** `plot_numor_data(hdf5_file, numor_ids[0])`

### get_data_for_ml
**Purpose:** Prepare and extract data for machine learning models.  
**Usage:** Fetches data in bulk or individually to feed into predictive models or statistical analyses.  
**Example:** `multiple_numor_data = get_data_for_ml(hdf5_file, numor_ids)`

### print_data_and_metadata
**Purpose:** Print detailed data and metadata for selected numors.  
**Usage:** Useful for detailed inspections of data and associated experimental parameters.  
**Example:** `print_data_and_metadata(hdf5_file, numor_ids)`
'''


In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

def open_hdf5(file_path):
    """ Open an HDF5 file and return the file object. """
    return h5py.File(file_path, 'r')

In [ ]:
import matplotlib.pyplot as plt

def plot_numor_data(hdf5_file, numor_id):
    """ Plot the data for a specific numor. Ensures only one numor is inputted. """
    # Check if numor_ids is a list and has exactly one item
    if isinstance(numor_id, int):
        numor_id = numor_id
    else:
        return "Error: You can only input a single numor here (e.g [107897])."

    group = hdf5_file[f'Numor{numor_id}']
    q_x = group['q_x (inverse angstrom)'][:]
    q_y = group['q_y (inverse angstrom)'][:]
    intensity = group['intensity (counts per standard monitor)'][:]

    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(q_x, q_y, c=intensity, cmap='jet', marker='s', s=40, edgecolors='none')
    cbar = plt.colorbar(scatter)
    #cbar.set_label(zlabel, rotation=270, labelpad=15)  # Adjust label orientation and position
    plt.xlabel('q_x (Å^{-1})')
    plt.ylabel('q_y (Å^{-1})')
    plt.title(f'Intensity Distribution for Numor {numor_id}')
    plt.show()


In [ ]:
def get_data_for_ml(hdf5_file, numor_ids):
    """ Extract data and metadata for ML/AI applications for one or multiple numors. """
    all_data = {}
    
    # Ensure numor_ids is a list for single and multiple numor handling
    if not isinstance(numor_ids, list):
        numor_ids = [numor_ids]
    
    for numor_id in numor_ids:
        group = hdf5_file[f'Numor{numor_id}']
        data = {
            'q_x': group['q_x (inverse angstrom)'][:],
            'q_y': group['q_y (inverse angstrom)'][:],
            'intensity': group['intensity (counts per standard monitor)'][:],
            'intensity_err': group['intensity_err (counts per standard monitor)'][:]
        }
        
        # Retrieve and include metadata in the dictionary
        metadata = {attr: group.attrs[attr] for attr in group.attrs}
        data.update(metadata)
        
        # Store data for this numor keyed by its ID
        all_data[numor_id] = data
    
    return all_data

In [ ]:
def print_data_and_metadata(hdf5_file, numor_ids):
    """ Print data and metadata for specified numors in a formatted manner.
    
    Args:
        hdf5_file (h5py.File): Open HDF5 file object.
        numor_ids (int or list): Single numor ID or list of numor IDs whose data and metadata are to be printed.
    """
    if isinstance(numor_ids, int):
        numor_ids = [numor_ids]  # Convert single numor ID to list for uniform processing

    for numor_id in numor_ids:
        group_name = f'Numor{numor_id}'
        if group_name in hdf5_file:
            group = hdf5_file[group_name]
            print(f"Data and Metadata for Numor {numor_id}:")
            print("-" * 60)  # Print a divider for better visual separation

            # Print datasets
            for dataset_name in ['q_x (inverse angstrom)', 'q_y (inverse angstrom)', 'intensity (counts per standard monitor)', 'intensity_err (counts per standard monitor)']:
                if dataset_name in group:
                    data_array = group[dataset_name][:]
                    print(f"{dataset_name} (sample points): {data_array.shape[0]}")
                    print(f"{dataset_name} (values):", data_array)

            # Print metadata
            print("Metadata:")
            for key, value in group.attrs.items():
                print(f"{key:25}: {value}")
            print("-" * 60)  # End divider
        else:
            print(f"Error: Numor {numor_id} not found in the HDF5 file.")
            print("-" * 60)


In [ ]:
# Open HDF5 File
hdf5_path = r"/Users/cadenmyers/billingelab/December_extracted_data.h5"
# r"/Users/cadenmyers/billingelab/dev/skyrmion-lattices/December_extracted_data.h5" #r"C:\Users\Nathan\OneDrive - nd.edu\Desktop\SANS Data\Experiments\PSI Cu2OSeO3 Corbino December 2023\Large Batch Data Extraction\Matlab-Grasp Programs\data\December_extracted_data.h5"
hdf5_file = open_hdf5(hdf5_path)
start_numor = 119546 #111001
end_numor =  119995#108370 #make same as start_numor if you only want to look at one numor
numor_ids = list(range(start_numor,end_numor+1))

# Example of Plotting Numor Data (can only use if specifying a single item in the list (e.q. numors_ids[0] or 115849))
plot_numor_data(hdf5_file, numor_ids[0])
#
# Example of Extracting Data for ML (builds a dictionary including the data and metadata)
single_numor_data = get_data_for_ml(hdf5_file, start_numor) # For a single numor

multiple_numor_data = get_data_for_ml(hdf5_file, numor_ids) # For multiple numors

# Example of Printing Data and Metadata
#print_data_and_metadata(hdf5_file, numor_ids[0])

In [ ]:
# Access matlab data
import scipy.io
import numpy as np

# Load file
file_path = '/Users/cadenmyers/billingelab/dev/skyrmion_lattices/Rotational_Velocity_121855_121914_Fit_Params.mat'
matlab_data = scipy.io.loadmat(file_path)
fitting_data = matlab_data['fitting_data']

# Extract target parameters
numor = fitting_data['numor'][0][0][0]
z0 = fitting_data['z0'][0][0][0]             # Background
i0 = fitting_data['i0'][0][0][0]             # Integrated Intensity
rx = fitting_data['rx'][0][0][0]             # Spot Radius
phi = fitting_data['phi'][0][0][0]           # Spot Angle
fwhm_r = fitting_data['fwhm_r'][0][0][0]     # FWHM Radial
fwhm_phi = fitting_data['fwhm_phi'][0][0][0] # FWHM Azimuthal
z0_err = fitting_data['z0_err'][0][0][0]
i0_err = fitting_data['i0_err'][0][0][0]
rx_err = fitting_data['rx_err'][0][0][0]
phi_err = fitting_data['phi_err'][0][0][0]
fwhm_r_err = fitting_data['fwhm_r_err'][0][0][0]
fwhm_phi_err = fitting_data['fwhm_phi_err'][0][0][0]
chi2 = fitting_data['chi2'][0][0][0]

# Check parameters
# print(phi)

In [ ]:
# Load inputs
# Adapt previous codes for loading single_numor_data
# Training data corresponds to 121855 56.5K -29mT.gif in Temp Sweep Movies
hdf5_file = open_hdf5(hdf5_path)
single_numor_data_all = []
for start_numor in numor:
    plot_numor_data(hdf5_file, start_numor)
    single_numor_data = get_data_for_ml(hdf5_file, start_numor)
    single_numor_data_all.append(single_numor_data)
# print(single_numor_data_all)

In [ ]:
# Extract input parameters
qx_all, qy_all, intensity_all = [], [], []
for i, start_numor in enumerate(numor):
    q_x = single_numor_data_all[i].get(start_numor).get('q_x')
    q_y = single_numor_data_all[i].get(start_numor).get('q_y')
    intensity = single_numor_data_all[i].get(start_numor).get('intensity')
    qx_all.append(q_x)
    qy_all.append(q_y)
    intensity_all.append(intensity)
qx_all = np.array(qx_all)
qy_all = np.array(qy_all)
intensity_all = np.array(intensity_all)

In [ ]:
# Save training data
np.savez('training_data.npz', intensity=intensity_all, phi=phi)